In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("attribution_allocation_student_data.csv")

df.head()

,convert_TF,touch_1,touch_2,touch_3,touch_4,touch_5,tier
0,True,email,direct,social,NaN,NaN,1
1,True,social,direct,organic_search,NaN,NaN,1
2,True,organic_search,display,social,NaN,NaN,1
3,True,social,direct,NaN,NaN,NaN,1
4,True,social,display,direct,NaN,NaN,1


In [49]:
# channel spend
channel_spend = pd.read_csv("channel_spend_student_data.csv")
channel_spend

,tier,spend by channel
0,1,"{'social': 50, 'organic_search': 0, 'referral'..."
1,2,"{'social': 100, 'organic_search': 0, 'referral..."
2,3,"{'social': 150, 'organic_search': 0, 'referral..."
3,total,"{'social': 300, 'organic_search': 0, 'referral..."


In [31]:
df.convert_TF.value_counts()

False    216034
True       2378
Name: convert_TF, dtype: int64

In [16]:
df_exposure = df[['touch_1', 'touch_2', 'touch_3', 'touch_4', 'touch_5']]

df_exposure['convert'] = df['convert_TF'].astype('int')

df_exposure['id'] = np.arange(len(df_exposure))

df_exposure = pd.melt(df_exposure, id_vars=['id', 'convert'], value_vars=['touch_1', 'touch_2', 'touch_3', 'touch_4', 'touch_5'],
       var_name = 'touch', value_name ='channel').sort_values(['id'])

df_exposure = df_exposure.dropna()

df_exposure = df_exposure.groupby('channel').convert.agg(['sum','count'])

df_exposure['convert_pct'] = df_exposure['sum'] / df_exposure['count']

df_exposure

/Users/Sandra/Documents/Assignments/1/AB_Testing/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,sum,count,convert_pct
channel,,,
direct,1725,143754,0.012000
display,1302,155700,0.008362
email,1092,85997,0.012698
organic_search,1884,158230,0.011907
paid_search,38,2528,0.015032
referral,36,2477,0.014534
social,1130,106998,0.010561


Suggests people aren't converting from Referral or Paid Search. Shows that the ads aren't reaching as many people.

## Part 1: Attribution

In [3]:
df_attribution = df[df['convert_TF'] == True].drop(['convert_TF'], axis = 1)

df_attribution['id'] = np.arange(len(df_attribution))

df_attribution = pd.melt(df_attribution, id_vars=['id','tier'], value_vars=['touch_1', 'touch_2', 'touch_3', 'touch_4', 'touch_5'],
       var_name = 'touch', value_name ='channel').sort_values(['id','touch'])

#drop nas
df_attribution =df_attribution.dropna(axis=0)

df_attribution['touch'] = df_attribution['touch'].str[-1].astype(int)

df_attribution['max_touch'] = df_attribution.groupby('id').touch.transform('max')

df_attribution['max_ind'] = np.where(df_attribution.touch == df_attribution.max_touch, 1, 0)

df_attribution.head()

,id,tier,touch,channel,max_touch,max_ind
0,0,1,1,email,3,0
2378,0,1,2,direct,3,0
4756,0,1,3,social,3,1
1,1,1,1,social,3,0
2379,1,1,2,direct,3,0


In [4]:
print("How many channels on avg exposed to: %.2f" % df_attribution.groupby('id').touch.agg('max').agg('mean'))

How many channels on avg exposed to: 3.03


In [5]:
# last interaction
df_lastint = df_attribution[df_attribution['max_ind'] == 1].channel.value_counts().reset_index().sort_values('index')

df_lastint.columns = ['channel', 'lastint_cnt']

df_lastint

,channel,lastint_cnt
1,direct,613
2,display,406
4,email,323
0,organic_search,662
5,paid_search,12
6,referral,10
3,social,352


In [6]:
# last non-direct
df_attribution_nodirect = df_attribution[df_attribution['channel'] != 'direct']

df_attribution_nodirect['max_ind'] = np.where(df_attribution_nodirect.touch == df_attribution_nodirect.groupby('id').touch.transform('max'), 1, 0)

df_lastndir = df_attribution_nodirect[df_attribution_nodirect['max_ind'] == 1].channel.value_counts().reset_index().sort_values('index')

df_lastndir.columns = ['channel', 'lastndir_cnt']

df_lastndir


/Users/Sandra/Documents/Assignments/1/AB_Testing/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,channel,lastndir_cnt
1,display,525
3,email,426
0,organic_search,896
4,paid_search,14
5,referral,14
2,social,454


In [7]:
# first interaction
df_firstint = df_attribution[df_attribution['touch'] == 1].channel.value_counts().reset_index().sort_values('index')

df_firstint.columns = ['channel', 'firstint_cnt']

df_firstint


,channel,firstint_cnt
1,direct,550
2,display,434
3,email,357
0,organic_search,675
5,paid_search,12
6,referral,10
4,social,340


In [8]:
# position based

def attribution(row):
    if row['max_touch'] == 1:
        val = 1
    elif row['max_touch'] == 2:
        val = 0.5
    elif row['touch'] == 1 or row['max_ind'] == 1:
        val = 0.4
    else:
        val = 0.2 / (int(row['max_touch']) - 2)
        
    return val
    
df_attribution['posbas_cnt'] = df_attribution.apply(attribution, axis=1)

#check if I did it correctly
df_attribution['check'] = df_attribution.groupby('id').posbas_cnt.transform(sum)
print("Check error: %d" % (sum(df_attribution['check'] != 1)))

df_posbas = df_attribution.groupby('channel').posbas_cnt.agg('sum').reset_index()

df_posbas


Check error: 0


,channel,posbas_cnt
0,direct,580.866667
1,display,416.700000
2,email,340.566667
3,organic_search,669.033333
4,paid_search,11.233333
5,referral,10.300000
6,social,349.300000


In [59]:
# join together

conv_attribution = df_lastint.merge(df_lastndir, on ='channel', how ='left').merge(df_firstint, on ='channel').merge(df_posbas, on ='channel')

conv_attribution = conv_attribution.fillna(0)

conv_attribution

,channel,lastint_cnt,lastndir_cnt,firstint_cnt,posbas_cnt
0,direct,613,0.0,550,580.866667
1,display,406,525.0,434,416.700000
2,email,323,426.0,357,340.566667
3,organic_search,662,896.0,675,669.033333
4,paid_search,12,14.0,12,11.233333
5,referral,10,14.0,10,10.300000
6,social,352,454.0,340,349.300000


In [64]:
cac = conv_attribution.copy()

cac.iloc[:,1:5] = 300 / cac.iloc[:,1:5]

cac

,channel,lastint_cnt,lastndir_cnt,firstint_cnt,posbas_cnt
0,direct,0.489396,inf,0.545455,0.516470
1,display,0.738916,0.571429,0.691244,0.719942
2,email,0.928793,0.704225,0.840336,0.880885
3,organic_search,0.453172,0.334821,0.444444,0.448408
4,paid_search,25.000000,21.428571,25.000000,26.706231
5,referral,30.000000,21.428571,30.000000,29.126214
6,social,0.852273,0.660793,0.882353,0.858861


**Comments on CAC:**
* A high proportion of conversions are coming from Organic Search, contrasting the performance of Paid Search. Rather than pulling money from Paid Search, it is recommended to review keyword strategy.

* Majority of conversions were from non-paid channels (Organic Search and Direct), regardless of method. While there’s no CAC associated with the channel, these conversions may be due to other channels. This suggests that there may be gaps in our ad tracking. It is recommended to review the current tracking between channels.

## Part 2: Allocation

In [76]:
def marginalcac(method):
    if method == "lastint":
        df_mcac = df_attribution[df_attribution['max_ind'] == 1].groupby(['tier', 'channel']).id.agg('count').reset_index()
    elif method == "lastint_ndir":
        df_mcac = df_attribution_nodirect[df_attribution_nodirect['max_ind'] == 1].groupby(['tier','channel']).id.agg('count').reset_index()
    elif method == "firstint":
        df_mcac = df_attribution[df_attribution['touch'] == 1].groupby(['tier','channel']).id.agg('count').reset_index()
    elif method == "posbas":
        df_mcac = df_attribution.groupby(['tier','channel']).posbas_cnt.agg('sum').reset_index()
        df_mcac.columns.values[2] = 'id'
    else:
        df = None 
    
    df_mcac['channel_spend'] = df_mcac['tier'] * 50
    df_mcac['channel_spend'] = np.where(df_mcac['channel'].isin(['direct','organic_search']), 0, df_mcac['channel_spend'])
    df_mcac['marginal_ac'] = df_mcac['id'] - df_mcac.groupby(['channel'])['id'].shift(1, fill_value = 0)
    df_mcac['marginal_spend'] = df_mcac['channel_spend'] - df_mcac.groupby(['channel'])['channel_spend'].shift(1, fill_value = 0)
    df_mcac['marginal_cac'] = df_mcac['marginal_spend'] / df_mcac['marginal_ac']
    df_mcac = df_mcac.pivot_table(index = 'tier', columns = 'channel', values ='marginal_cac', aggfunc ='sum')
    
    return df_mcac

In [77]:
marginalcac("lastint")

channel,direct,display,email,organic_search,paid_search,referral,social
tier,,,,,,,
1,0.0,0.694444,0.769231,0.0,25.000000,50.0,0.862069
2,0.0,0.574713,1.470588,0.0,inf,50.0,0.704225
3,0.0,3.125000,0.833333,0.0,8.333333,10.0,1.388889


In [71]:
marginalcac("lastint_ndir")

channel,display,email,organic_search,paid_search,referral,social
tier,,,,,,
1,0.500000,0.625000,0.0,16.666667,25.0,0.602410
2,0.505051,1.000000,0.0,-50.000000,25.0,0.602410
3,1.851852,0.581395,0.0,7.142857,12.5,1.282051


In [72]:
marginalcac("firstint")

channel,direct,display,email,organic_search,paid_search,referral,social
tier,,,,,,,
1,0.0,0.543478,0.746269,0.0,25.000000,25.000000,0.704225
2,0.0,0.666667,1.020408,0.0,16.666667,-50.000000,1.063830
3,0.0,6.250000,0.862069,0.0,inf,8.333333,1.515152


In [78]:
marginalcac("posbas")

channel,direct,display,email,organic_search,paid_search,referral,social
tier,,,,,,,
1,0.0,0.599520,0.774793,0.0,24.590164,35.714286,0.759494
2,0.0,0.667557,1.082251,0.0,40.540541,71.428571,0.827815
3,0.0,2.994012,0.916310,0.0,18.750000,10.638298,1.612903
